In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import datetime
from bs4 import BeautifulSoup
import cv2
url = 'https://pportal.hpa.gov.tw/Web/Notice.aspx'

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.maximize_window()
time.sleep(1)
driver.find_element('xpath','//*[@id="ctl00_pnlLogin"]/button').click()
print("網址設定成功")


C:\Users\dl\AppData\Local\Temp\ipykernel_17804\1202833423.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


網址設定成功


In [2]:
driver.save_screenshot('screen.jpg')
print("screenshot down!")

C:\ProgramData\anaconda3\envs\myenv\lib\site-packages\selenium\webdriver\remote\webdriver.py:889: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn(


screenshot down!


In [3]:
element =driver.find_element('xpath','//*[@id="imgCheckCode"]')#尋找驗證碼的位置

In [4]:
element.location

{'x': 957, 'y': 344}

In [5]:
element.size

{'height': 31, 'width': 70}

In [6]:
left = element.location['x']
top = element.location['y']
right = left + element.size['width']
bottom = top + element.size['height']


In [7]:
left, right, top, bottom

(957, 1027, 344, 375)

In [8]:
from PIL import Image
img  = Image.open('screen.jpg')

In [9]:
img = img.crop((left, top, right, bottom))

In [10]:
img = img.convert("RGB") # 處理RBGA的通道數問題，Bug:OSError: cannot write mode RGBA as JPEG <----JPEG不存在A通道

img

In [11]:
img.size

(70, 31)

In [12]:
img.save('numbercheck.jpg', 'jpeg')

In [13]:
img = img.convert("L")  # 將圖片轉成灰階
pix_img = img.load()
w, h = img.size
threshold = 205 #像素閥值
for y in range(h):    #for迴圈將每個像素點進行判斷
    for x in range(w):
        if pix_img[x, y] < threshold:
            pix_img[x, y] = 0
        else:
            pix_img[x, y] = 255

In [14]:
# 將經過閥值處理的像素重新生成圖片
data = img.getdata()
w, h = img.size
black_point = 0
for x in range(1, w - 1):
    for y in range(1, h - 1):
        mid_pixel = data[w * y + x]
        if mid_pixel < 50:
            top_pixel = data[w * (y - 1) + x]
            left_pixel = data[w * y + (x - 1)]
            down_pixel = data[w * (y + 1) + x]
            right_pixel = data[w * y + (x + 1)]
            if top_pixel < 10:
                black_point += 1
            if left_pixel < 10:
                black_point += 1
            if down_pixel < 10:
                black_point += 1
            if right_pixel < 10:
                black_point += 1
            if black_point < 1:
                img.putpixel((x, y), 255)
            black_point = 0

In [15]:
w, h

(70, 31)

In [16]:
img

In [183]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\pytess\tesseract.exe' #設定位址
print("pytesseract 設定ok")
result = pytesseract.image_to_string(img)
regex = '\d+'
result = result.replace(" ", "") #去除空格，以免造成有'29 1316'這種中間空一格的情形
print(result)

pytesseract 設定ok



In [146]:
numbers

[]